# Standalone Model Inference Script

This notebook allows you to run inference with a pre-trained PyTorch model (FNN, GRU, or LSTM) from the `vestim` project on a new dataset.

**Instructions:**
1.  Fill in the `JOB_FOLDER_PATH`, `MODEL_FOLDER_PATH`, and `TEST_DATA_PATH` in the 'Configuration Parameters' cell below.
2.  The script will automatically find the correct exported model, and scaler from the job folder.
3.  If your model requires features that were generated during training (e.g., filtered columns), use the optional 'Data Augmentation' section to create them.
4.  The script will validate that all necessary columns are present and provide a clear error if columns are missing.
5.  Run the cells sequentially.

## 1. Imports

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import joblib
import json
import os
from datetime import datetime
from scipy.signal import butter, filtfilt

## 2. Model Class Definitions

These are the model definitions copied from the `vestim` project to ensure this notebook is self-contained. Make sure these match the definitions used during training.

In [ ]:
class FNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_layer_sizes_str, activation_str='ReLU', dropout_prob=0.0):
        super(FNNModel, self).__init__()
        hidden_layer_sizes = [int(size.strip()) for size in hidden_layer_sizes_str.split(',')]
        
        layers = []
        current_dim = input_size
        
        activation_fn = getattr(nn, activation_str, nn.ReLU)

        for hidden_dim in hidden_layer_sizes:
            layers.append(nn.Linear(current_dim, hidden_dim))
            layers.append(activation_fn())
            if dropout_prob > 0:
                layers.append(nn.Dropout(dropout_prob))
            current_dim = hidden_dim
        
        layers.append(nn.Linear(current_dim, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        if x.ndim == 3:
            x = x.view(x.size(0), -1)
        return self.network(x)

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_units, num_layers, output_size=1, device='cpu'):
        super(GRUModel, self).__init__()
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.device = device
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_units, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_units, output_size)

    def forward(self, x, h_0=None):
        if h_0 is None:
            h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_units).to(self.device)
        out, _ = self.gru(x, h_0)
        out = self.fc(out[:, -1, :])
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_units, num_layers, output_size=1, device='cpu'):
        super(LSTMModel, self).__init__()
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.device = device
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_units, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_units, output_size)

    def forward(self, x, h_s=None, h_c=None):
        if h_s is None or h_c is None:
            h_s = torch.zeros(self.num_layers, x.size(0), self.hidden_units).to(self.device)
            h_c = torch.zeros(self.num_layers, x.size(0), self.hidden_units).to(self.device)
        out, _ = self.lstm(x, (h_s, h_c))
        out = self.fc(out[:, -1, :])
        return out

## 3. Configuration Parameters

In [ ]:
# --- REQUIRED PATHS ---
JOB_FOLDER_PATH = "output/job_20250902-141234"      # Path to the root job folder from your training run
MODEL_FOLDER_PATH = "output/job_20250902-141234/models/FNN_64_32/B4096_LR_SLR_VP120_rep_3" # Path to the specific model folder containing 'best_model_export.pt'
TEST_DATA_PATH = "path/to/your/test_data.csv"      # Path to your new test CSV file
# --- END OF REQUIRED PATHS ---

# --- DERIVED PATHS (DO NOT MODIFY) ---
EXPORTED_MODEL_PATH = os.path.join(MODEL_FOLDER_PATH, 'best_model_export.pt')
SCALER_PATH = os.path.join(JOB_FOLDER_PATH, 'scalers', 'augmentation_scalers.joblib')
JOB_METADATA_PATH = os.path.join(JOB_FOLDER_PATH, 'job_metadata.json')

# Device Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

## 4. Helper Functions

In [ ]:
def apply_butterworth_filter(df, column_name, new_column_name, filter_order, sampling_freq, corner_freq):
    """Applies a Butterworth low-pass filter to a column and adds it to the DataFrame."""
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")
    
    # Butterworth filter design
    nyquist = 0.5 * sampling_freq
    if corner_freq >= nyquist:
        raise ValueError(f"Corner frequency ({corner_freq}Hz) must be less than the Nyquist frequency ({nyquist}Hz). Adjust your filter settings.")
        
    normal_corner = corner_freq / nyquist
    b, a = butter(filter_order, normal_corner, btype='low', analog=False)
    
    # Apply the filter
    filtered_data = filtfilt(b, a, df[column_name])
    
    # Add the new column to the DataFrame
    df[new_column_name] = filtered_data
    print(f"Created filtered column '{new_column_name}' from '{column_name}'.")
    return df

def create_sequences(data, lookback):
    """Creates sequences from a numpy array."""
    X = []
    for i in range(len(data) - lookback + 1):
        X.append(data[i:(i + lookback)])
    return np.array(X)

def inverse_transform_single_column(data, scaler, column_name, all_columns):
    """Inverse transforms a single column of data using a multi-feature scaler."""
    try:
        col_index = all_columns.index(column_name)
    except ValueError:
        raise ValueError(f"Column '{column_name}' not found in the list of scaled columns.")
    
    dummy_array = np.zeros((len(data), len(all_columns)))
    dummy_array[:, col_index] = data.flatten()
    
    inversed_data = scaler.inverse_transform(dummy_array)
    return inversed_data[:, col_index]

## 5. Main Inference Logic

In [ ]:
# --- 1. Load Model Checkpoint and Configuration ---
print("--- Loading Model Checkpoint and Configuration ---")
try:
    checkpoint = torch.load(EXPORTED_MODEL_PATH, map_location=DEVICE)
    print(f"Successfully loaded model checkpoint from {EXPORTED_MODEL_PATH}")
except Exception as e:
    raise IOError(f"Error loading model checkpoint file: {e}")

model_hyperparams = checkpoint.get('hyperparams', {})
data_config = checkpoint.get('data_config', {})
MODEL_TYPE = checkpoint.get('model_type')
FEATURE_COLUMNS = data_config.get('feature_columns')
TARGET_COLUMN = data_config.get('target_column')
LOOKBACK = int(data_config.get('lookback', 0))
TRAINING_METHOD = checkpoint.get('model_metadata', {}).get('training_method')

# --- 2. Load Test Data ---
print("\n--- Loading Test Data ---")
try:
    test_df = pd.read_csv(TEST_DATA_PATH)
    print(f"Successfully loaded test data from {TEST_DATA_PATH}")
except Exception as e:
    raise IOError(f"Error loading test data file: {e}")

# --- 3. Optional: Data Augmentation ---
print("\n--- Optional: Data Augmentation ---")
# EXAMPLE: If your model was trained on a filtered 'Power' column, create it here.
# You can call this function multiple times for different columns.
# test_df = apply_butterworth_filter(
#     df=test_df,
#     column_name='Power', 
#     new_column_name='Watts_fltr_3e-3', 
#     filter_order=2, 
#     sampling_freq=10, # Hz
#     corner_freq=0.003 # Hz
# )
print("Skipping data augmentation. Uncomment and edit the example above if needed.")

# --- 4. Load Scaler and Validate Columns ---
print("\n--- Loading Scaler and Validating Columns ---")
scaler = None
normalized_columns = []

try:
    with open(JOB_METADATA_PATH, 'r') as f:
        job_metadata = json.load(f)
    if job_metadata.get('normalization_applied', False):
        scaler = joblib.load(SCALER_PATH)
        normalized_columns = job_metadata.get('normalized_columns')
        print(f"Successfully loaded scaler from {SCALER_PATH}")
        
        missing_scaler_cols = [col for col in normalized_columns if col not in test_df.columns]
        if missing_scaler_cols:
            error_message = (f"\n\nCRITICAL ERROR: The test data is missing columns required for normalization: {missing_scaler_cols}.\n\n" \
                           "These columns were likely generated during data augmentation in the original training run.\n\n" \
                           "Please use the 'Data Augmentation' cell above to create these columns before proceeding.")
            raise ValueError(error_message)
        print("All columns required for normalization are present.")
    else:
        print("Normalization was not applied during training. No scaler loaded.")
except FileNotFoundError:
    print("job_metadata.json or scaler not found. Assuming no normalization was applied.")
except Exception as e:
    print(f"Warning: Could not load scaler or metadata. Error: {e}")

# --- 5. Prepare Data for Inference ---
print("\n--- Preparing Data for Inference ---")
missing_feature_cols = [col for col in FEATURE_COLUMNS if col not in test_df.columns]
if missing_feature_cols:
    raise ValueError(f"Error: The test data is missing required model feature columns: {missing_feature_cols}")
print("All required model feature columns are present.")

data_to_process = test_df[FEATURE_COLUMNS].copy()
if scaler:
    data_to_process[normalized_columns] = scaler.transform(data_to_process[normalized_columns])
    print("Test data normalized using the loaded scaler.")

if TRAINING_METHOD == 'WholeSequenceFNN':
    X_test_np = data_to_process.values
    X_test = torch.tensor(X_test_np, dtype=torch.float32).to(DEVICE)
    print("Data prepared for WholeSequenceFNN (no sequences).")
else:
    if len(data_to_process) < LOOKBACK:
        raise ValueError(f"Test data length ({len(data_to_process)}) is less than the model's lookback window ({LOOKBACK}).")
    X_test_np = create_sequences(data_to_process.values, LOOKBACK)
    X_test = torch.tensor(X_test_np, dtype=torch.float32).to(DEVICE)
    print(f"Data prepared with sequences of lookback {LOOKBACK}.")

# --- 6. Load Model ---
print("\n--- Loading Model ---")
input_size = len(FEATURE_COLUMNS)

if MODEL_TYPE == 'FNN':
    model_input_size = input_size * LOOKBACK if TRAINING_METHOD != 'WholeSequenceFNN' else input_size
    model = FNNModel(
        input_size=model_input_size,
        output_size=model_hyperparams['output_size'],
        hidden_layer_sizes_str=model_hyperparams['hidden_layer_sizes'],
        activation_str=model_hyperparams.get('activation', 'ReLU'),
        dropout_prob=float(model_hyperparams.get('dropout_prob', 0.0))
    )
elif MODEL_TYPE == 'LSTM':
    model = LSTMModel(
        input_size=input_size,
        hidden_units=int(model_hyperparams['hidden_size']),
        num_layers=int(model_hyperparams['num_layers']),
        device=DEVICE
    )
elif MODEL_TYPE == 'GRU':
    model = GRUModel(
        input_size=input_size,
        hidden_units=int(model_hyperparams['hidden_size']),
        num_layers=int(model_hyperparams['num_layers']),
        device=DEVICE
    )
else:
    raise ValueError(f"Unsupported model type: {MODEL_TYPE}")

model.load_state_dict(checkpoint['state_dict'])
model.to(DEVICE)
model.eval()
print(f"Successfully instantiated and loaded model state.")

# --- 7. Run Inference ---
print("\n--- Running Inference ---")
with torch.no_grad():
    predictions_normalized = model(X_test)
predictions_normalized = predictions_normalized.cpu().numpy()
print(f"Inference complete. Generated {len(predictions_normalized)} predictions.")

# --- 8. Denormalize Predictions ---
print("\n--- Denormalizing Predictions ---")
if scaler:
    predictions_final = inverse_transform_single_column(predictions_normalized, scaler, TARGET_COLUMN, normalized_columns)
    print(f"Predictions denormalized for target column '{TARGET_COLUMN}'.")
else:
    predictions_final = predictions_normalized.flatten()
    print("No scaler was used; predictions are in the original scale.")

# --- 9. Save Predictions ---
print("\n--- Saving Predictions ---")
output_df = pd.DataFrame()
if TRAINING_METHOD == 'WholeSequenceFNN':
    output_df = test_df.copy()
    output_df['Predicted_Value'] = predictions_final
else:
    prediction_start_index = LOOKBACK - 1
    output_df = test_df.iloc[prediction_start_index:].copy()
    output_df = output_df.iloc[:len(predictions_final)]
    output_df['Predicted_Value'] = predictions_final

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name_part = os.path.basename(MODEL_FOLDER_PATH)
output_filename = os.path.join(os.path.dirname(TEST_DATA_PATH), f"predictions_{model_name_part}_{timestamp}.csv")

try:
    output_df.to_csv(output_filename, index=False)
    print(f"Successfully saved predictions to: {output_filename}")
except Exception as e:
    print(f"Error saving predictions file: {e}")
